In [1]:
!pip install youtube-transcript-api transformers python-docx fpdf

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.6/484.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.5 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=e00aeb59ef07b0666e8fb74b1ebf0fd061e485d4019b48986db1da5458ce265e
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [2]:
import re
from textwrap import wrap
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
from transformers import pipeline
from fpdf import FPDF
from docx import Document
from google.colab import files
from youtube_transcript_api.formatters import TextFormatter
from urllib.parse import urlparse, parse_qs


In [3]:
def extract_video_id(url):
    pattern = r"(?:v=|\/)([0-9A-Za-z_-]{11})"
    match = re.search(pattern, url)
    return match.group(1) if match else None

video_url = input("📎 Paste the YouTube video link: ")
video_id = extract_video_id(video_url)
if not video_id:
    print("❌ Invalid YouTube URL. Could not extract video ID.")
else:
    # 🌐 Ask for preferred language
    lang_input = input("🌐 Enter language code (default 'en'): ").strip().lower() or "en"
    preferred_languages = [lang_input]

    try:
        # 🎥 Fetch transcript
        transcript_data = YouTubeTranscriptApi.get_transcript(video_id, languages=preferred_languages)
        print(f"✅ Transcript fetched successfully in: {lang_input}")

        # 📄 Join full transcript text
        full_text = "\n".join([t['text'] for t in transcript_data])
        chunks = wrap(full_text, width=1000)
        print(f"📑 Total Chapters/Chunks: {len(chunks)}")

        # 🤖 Load summarizer
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

        chapter_notes = []

        for i, chunk in enumerate(chunks):
            try:
                summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
            except Exception as e:
                summary = f"[⚠️ Error summarizing chunk {i+1}: {str(e)}]"
            chapter_notes.append(f"\n📘 Chapter {i+1}:\n{summary}")

        # 📚 Output summaries
        for note in chapter_notes:
            print(note)

    except TranscriptsDisabled:
        print("❌ Captions are disabled for this video.")
    except NoTranscriptFound:
        print(f"❌ No transcript found in {preferred_languages}. Try another like 'en', 'hi', etc.")
    except Exception as e:
        print(f"❌ Unexpected error occurred: {str(e)}")

📎 Paste the YouTube video link: https://youtu.be/z7TOTY6uOPA?si=zHGdIIkRsEQ0JGC4
🌐 Enter language code (default 'en'): en
❌ Unexpected error occurred: type object 'YouTubeTranscriptApi' has no attribute 'get_transcript'
